In [1]:
import syft as sy
import numpy as np
import pandas as pd
from syft.core.adp.entity_list import EntityList
from time import time

In [2]:
t0 = time()
df = pd.read_parquet("/home/ruchi/1B_rows_dataset_sample.parquet")
# df = pd.read_parquet("../Ishan/1M_rows_dataset_sample.parquet")
tf = time() - t0

print(f"Time taken to read file with {df.shape[0]/1e6} million rows : {tf} seconds")

Time taken to read file with 1000.0 million rows : 92.12655973434448 seconds


In [3]:
scale = 1000  # This is put here to reduce the size of the cache :)


t0 = time()
impressions = df['impressions'].to_numpy(np.int64) // scale
data_subjects = EntityList.from_series(df['user_id'])
tf = time() - t0

print(f"Time taken to create inputs for Syft Tensor: {tf} seconds")

Time taken to create inputs for Syft Tensor: 50.22229266166687 seconds


In [4]:
t0 = time()
tweets_data = sy.Tensor(impressions).private(min_val=700_000/scale, max_val=20e6/scale, entities = data_subjects,ndept=True)  # RUn this for 1 billion rows
# tweets_data = sy.Tensor(impressions).private(min_val=500/scale, max_val=25000/scale, entities = data_subjects,ndept=True)  # Run this for 1 million rows
tf = time() - t0

print(f"Time make Private Syft Tensor: {tf} seconds")

Time make Private Syft Tensor: 2.2126710414886475 seconds


In [5]:
domain_node = sy.login(email="info@openmined.org",password="changethis",port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into kind_goodfellow... done!


In [6]:
from syft.core.node.common.node_service.user_manager.user_messages import (
    UpdateUserMessage,
)

# Upgrade admins budget
content = {"user_id": 1, "budget": 9_999_999}
domain_node._perform_grid_request(grid_msg=UpdateUserMessage, content=content)

domain_node.privacy_budget

9999999.0

In [7]:
%%time
name = f"Tweets data- {time()}"

domain_node.load_dataset(
    assets={name: tweets_data},
    name=name,
    description=" Tweets- 1B rows",
    use_blob_storage=True
)

Loading dataset... checking asset types...                                                                                                                                    

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/ruchi/PySyft
Uploading Tweets data- 1648062991.2597036: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:08<00:00,  1.24s/it]


Loading dataset... uploading... SUCCESS!                        

Run <your client variable>.datasets to see your new dataset loaded into your machine!
CPU times: user 40.9 s, sys: 11 s, total: 51.8 s
Wall time: 1min 10s


In [8]:
domain_node.datasets

Idx,Name,Description,Assets,Id
[0],Tweets data- 1648062991.2597036,Tweets- 1B rows,"[""Tweets data- 1648062991.2597036""] ->",69a7c9fe-f157-483a-8110-236823e27465


In [9]:
data = domain_node.datasets[-1][name]

In [64]:
res = tweets_data.child.sum()

In [66]:
ser = sy.serialize(res, to_bytes=True)

700000000000.0


In [74]:
ser[:200]

b'\x11\x056\xf7\xd4e\x01\x01\x90\xd0\x03\xff\x01\x90\xd0\x03\x02 \xa1\x07\x00\x0fc\xc6\xa0\x0cP\x03\n\xf8\x0bh_dB\xfc@\xe5\x9c0\xb2B\x01\x01\x11%\xba\x11-\x0e\x11i\x0e\x11uF\x11\x9d\x0eQP\x01\x01Q\\\x01\x01Q\x84\x01\x01Q\x90\x01\x01\x11\x99R\xffcapnp:Ga\x02mmaTensor:capnp\x001\x01\x02\x03\xff(\xb5/\xfd \xc8\xbd\x02\x0b\x00\x02\x84\r\x19\xc05s\xff\xff\xae\xff]\xd6\xff\xff\xff\xde\x81F\xbax\x93\xb2_\x92\x94\xdd$\x05\x17IZ\x00\xcfN\xa0\x8e\x1b\x81m\r"\x93UW"\xf4\xeb\xaa\x0eS\xe4\xda\x1e\xb8\xbb\x0b\r\x00\x01\x07\xab\x08\xc2\xc5\x10\xfa\xc0\x8a\x1c\x06[I\x00\xf1>%\x88\xcf\xd1G'

In [73]:
deser = sy.deserialize(ser, from_bytes=True)

failed capnp deserialize subsection not found


ValueError: subsection not found

In [63]:
# sy.serialize(tweets_data.child.sum())

700000000000.0


KeyboardInterrupt: 

In [10]:
%%time
sum_result = data.sum()
sum_result.block

CPU times: user 2.41 s, sys: 141 ms, total: 2.55 s
Wall time: 1min 40s


<TensorPointer -> kind_goodfellow:0dcbca0a2b1942258a9fed387d4bb3d1>

In [62]:
s = sum_result.get_copy()

failed capnp deserialize subsection not found
failed capnp deserialize subsection not found


ValueError: subsection not found

In [11]:
sum_result.exists

True

In [60]:
sum_result.id_at_location

<UID: 7ca73ddafc00414ab9d170f641bdca5e>

In [12]:
published_result = sum_result.publish(sigma=100)

In [16]:
published_result.exists

True

In [17]:
# published_result

<FloatPointer -> hardcore_gardner:14669bffb0d34f63a93322fab4ef416f>

In [21]:
# published_result.request()

In [22]:
# domain_node.requests

,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Jane Doe,info@openmined.org,Owner,DATA,pending,,<UID: 2352122b71e24e6c9826501d7eb3cd28>,<UID: 14669bffb0d34f63a93322fab4ef416f>,[],None,None


In [23]:
# domain_node.requests[-1].accept()

In [24]:
# domain_node.requests

,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Jane Doe,info@openmined.org,Owner,DATA,accepted,,<UID: 2352122b71e24e6c9826501d7eb3cd28>,<UID: 14669bffb0d34f63a93322fab4ef416f>,[],None,None


In [25]:
# result_of_our_hard_labour = published_result.get()

# WE GOT AN OVERFLOW ERROR
- doesn't occur with 1M rows
- publish had completed, tho it took 602 seconds

In [26]:
# result_of_our_hard_labour

array(-2147483648, dtype=int32)

In [22]:
# impressions.sum()

11941995258

In [23]:
def percentage_error(true_value, noisy_value):
    return (true_value - noisy_value)/true_value * 100

In [24]:
# percentage_error(true_value=impressions.sum(), noisy_value=result_of_our_hard_labour)

96.35989431741909

In [13]:
# (1644527104 - 11941995258)/11941995258 * 100

-86.22904239642598

In [14]:
# 11941995258 /1e6

11941.995258

In [31]:
def calculate_bounds_for_mechanism(
     min_val_array, max_val_array
):
    """Calculates the squared L2 norm values needed to create a Mechanism, and calculate
    privacy budget + spend. If you calculate the privacy budget spend with the worst
    case bound, you can show this number to the DS. If you calculate it with the
    regular value (the value computed below when public_only = False, you cannot show
    the privacy budget to the DS because this violates privacy."""
    l2_norm_min = np.sqrt(np.sum(np.square(min_val_array)))
    l2_norm_max = np.sqrt(np.sum(np.square(max_val_array)))
    return l2_norm_min, l2_norm_max

In [32]:
def _get_batch_rdp_constants(
    sigma, scale, min_val, max_val,  L=1
) -> np.ndarray:
    min_val = min_val/scale
    max_val = max_val/scale
    # print(min_val, max_val)
    l2_norm_min, l2_norm_max = calculate_bounds_for_mechanism(min_val, max_val)
    
    # use the indices to get a "batch" of the full ledger. this is the only part
    # of the ledger we care about (the entries corresponding to specific entities)
    squared_Ls = L**2
    squared_sigma = sigma**2
    squared_L2_norms_min = l2_norm_min**2
    squared_L2_norms_max = l2_norm_max**2
    
    constant_min = (
        squared_Ls * squared_L2_norms_min / (2 * squared_sigma)
    )
    constant_max = (
        squared_Ls * squared_L2_norms_max / (2 * squared_sigma)
    )
    
#     constant = np.bincount(batch_entity_ids, weights=constant).take(
#         entity_ids_query
#     )
    # # update our serialized format with the calculated constants
    # self._rdp_constants = np.concatenate([self._rdp_constants, constant])
    # self._entity_ids_query = np.concatenate(
    #     [self._entity_ids_query, entity_ids_query]
    # )
    return constant_min, constant_max

In [48]:
#_get_batch_rdp_constants(sigma=100, scale=1000, min_val=700_000, max_val=20e6)

(24.5, 20000.0)